In [25]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "root"
password = "gD70TDm1ON"

#db = AnimalShelter(username, password)
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read())

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# DEBUG
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU - Global Rain Dashboard - Thomas Keller'))),
    html.Hr(),
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    dcc.RadioItems(
                        id='radio-buttons',
                        options=[
                            {'label': 'Reset', 'value': 'option0'},
                            {'label': 'Cat', 'value': 'option1'},
                            {'label': 'Dog', 'value': 'option2'},
                            {'label': 'Disaster Rescue', 'value': 'option3'},
                            {'label': 'Wilderness Rescue', 'value': 'option4'},
                            {'label': 'Water Rescue', 'value': 'option5'},
                        ],
                        value='option0'
                    ),
                    html.Div(id='radio-output')
                ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=15
                        ),
     dl.Map(
            [dl.TileLayer(), dl.Marker(position=(30.75,-97.48))],
            style={"width": "100%", "height": "500px"},
            id='Map-id',
            center=(30.75,-97.48),
            zoom=10,
        ),
    html.H1("Query Breed Distribution"),
        dcc.Graph(
            id='pie-chart',
        ),
    html.Br(),
    html.Hr()
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', "data"),
             [Input('radio-buttons', 'value'),
             ])
def on_click(value):
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    #uses radio buttons to determine filters such as cat, dog, rescue, and others
    if value == 'option0':
        df = pd.DataFrame.from_records(db.read({}))
    
    elif value == 'option1':
        df = pd.DataFrame.from_records(db.read({"animal_type" : "Cat"}))
        
    elif value == 'option2':
        df = pd.DataFrame.from_records(db.read({"animal_type" : "Dog"}))
        
    elif value == 'option3':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                                                "breed":{"$in":["Doberman Pinscher","German Shepherd",
                                                                "Golden Retriever","Bloodhound", "Rottweiler"]},
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":20},
                                                "age_upon_outcome_in_weeks":{"$lte":300}}))
        
    elif value == 'option4':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                                                "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif value == 'option5':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                                                "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                                                "sex_upon_outcome":"Intact Female",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$lte":156}}))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')


# Define the callback function
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "derived_viewport_data")],
)
def update_output(data):
    labels = [row["breed"] for row in data]
    occurrences = dict(Counter(labels))
    data = [go.Pie(labels=labels, values=occurrences)]
    
    layout = go.Layout(
        title='Breeds',
        height=400
    )
    
    return {'data': data, 'layout': layout}

    

@app.callback(
    Output("Map-id", "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
     # Because we only allow single row selection, the list can 
     # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:24681/
